https://ieeexplore.ieee.org/document/8141873

https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.metrics import confusion_matrix
import numpy as np

In [3]:
from transplant.data.learningset import Learningset
from transplant.config import *
learningset = Learningset()

X_train, X_test, y_train, y_test, X_col = learningset.get_data_merged_dynamic_flatten_full("One_Hot", False)
#X_train, X_test, y_train, y_test, X_col = learningset.get_data_merged_dynamic_flatten_full("cls", False)

In [4]:
#y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)

In [5]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((241, 133), (101, 133), (241, 2), (101, 2))

In [6]:
y_train.sum(axis=0)

array([137, 104])

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout


Using TensorFlow backend.


In [10]:
model = Sequential()
model.add(Dense(130, activation='relu', input_dim=(133))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train, epochs=10, batch_size=50)

Epoch 1/10
241/241 [==============================] - 0s 2ms/step - loss: 9.1126 - acc: 0.4315
Epoch 2/10
241/241 [==============================] - 0s 83us/step - loss: 9.1126 - acc: 0.4315
Epoch 3/10
241/241 [==============================] - 0s 100us/step - loss: 9.1126 - acc: 0.4315
Epoch 4/10
241/241 [==============================] - 0s 133us/step - loss: 9.1126 - acc: 0.4315
Epoch 5/10
241/241 [==============================] - 0s 108us/step - loss: 9.1126 - acc: 0.4315
Epoch 6/10
241/241 [==============================] - 0s 108us/step - loss: 9.1126 - acc: 0.4315
Epoch 7/10
241/241 [==============================] - ETA: 0s - loss: 7.3739 - acc: 0.540 - 0s 116us/step - loss: 9.1126 - acc: 0.4315
Epoch 8/10
241/241 [==============================] - 0s 116us/step - loss: 9.1126 - acc: 0.4315
Epoch 9/10
241/241 [==============================] - 0s 141us/step - loss: 9.1126 - acc: 0.4315
Epoch 10/10
241/241 [==============================] - 0s 124us/step - loss: 9.1126 - acc: 0

In [12]:
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=28)

101/101 [==============================] - 0s 891us/step


In [ ]:
classes = model.predict(X_test, batch_size=128)

In [ ]:
def argmax(x):
    return np.array(list(map(np.argmax, x)))

In [ ]:
confusion_matrix(argmax(y_test),argmax(classes))

In [ ]:
y_test.shape

In [ ]:
argmax(classes)

# Tests

In [ ]:
def get_data_merged_dynamic(self, target_format="cls"):


        from transplant.data.learningset import Learningset
        learningset = Learningset()
        train_static_0, test_static_0 = learningset.get_static_filled()

        train_dynamic_0, test_dynamic_0 = dataset.get_dynamic()

        mean_dynamic_train = train_dynamic_0.groupby(['id_patient']).mean().mean()
        
        train_dynamic_0 = train_dynamic_0.fillna(mean_dynamic_train)
        test_dynamic_0 = test_dynamic_0.fillna(mean_dynamic_train)

        def add_start_end_length_op_to_static(X_stat, X_dyn):
            grouped_time = X_dyn.groupby(['id_patient'])['time']

            time_start_df = grouped_time.first().to_frame()
            time_start_df.columns = ['start_operation']
            time_start_df['id_patient'] = time_start_df.index

            X_return = pd.merge(X_stat, time_start_df, on='id_patient')

            time_ends_df = grouped_time.last().to_frame()
            time_ends_df.columns = ['ends_operation']
            time_ends_df['id_patient'] = time_ends_df.index

            X_return = pd.merge(X_return, time_ends_df, on='id_patient')

            X_return['length_op'] = X_return['ends_operation'] - X_return['start_operation']

            return X_return

        train_static_1 = add_start_end_length_op_to_static(train_static_0, train_dynamic_0)
        test_static_1 = add_start_end_length_op_to_static(test_static_0, test_dynamic_0)

        def get_timeseries_in_array(X_stat, X_dyn, dyn_to_drop=['id_patient', 'time']):

            X_return = X_stat

            grouped = X_dyn.groupby(['id_patient'])

            list_time_serie_col = X_dyn.drop(dyn_to_drop, axis=1).columns

            for i in list_time_serie_col:
                df_muette = grouped[i].apply(np.array).to_frame()
                df_muette['id_patient'] = df_muette.index
                X_return = pd.merge(X_return, df_muette, on='id_patient')

            return X_return
            


        train_glob = get_timeseries_in_array(train_static_1, train_dynamic_0)
        test_glob = get_timeseries_in_array(test_static_1, test_dynamic_0)
        



        dic_to_One_Hot = {0: [1, 0], 1: [0, 1]}

        y_train_cls = np.array(train_glob['target'])
        y_train_hot = np.array(list(train_glob['target'].map(dic_to_One_Hot)))

        y_test_cls = np.array(test_glob['target'])
        y_test_hot = np.array(list(test_glob['target'].map(dic_to_One_Hot)))


        X_train = np.array(train_glob.drop(['target'], axis=1))
        X_test = np.array(test_glob.drop(['target'], axis=1))

        # Return
        if target_format == "cls":
            return X_train, X_test, y_train_cls, y_test_cls, train_glob.drop(['target'], axis=1).columns

        if target_format == "One_Hot":
            return X_train, X_test, y_train_hot, y_test_hot, train_glob.drop(['target'], axis=1).columns